### Decay Regression 

In [ ]:
def decay_regress(x: np.ndarray, y: np.ndarray, *, prior_mean: np.ndarray,
                  prior_covar: np.ndarray,
                  decay_scale: float | np.ndarray = np.Inf,
                  y_decay_scale: float | None = None) -> np.ndarray:
    """
    Parallel rolling time-series regression with decay and a prior mean and
    prior covariance matrix.
    The N time-series are fit independently (in parallel), but share priors.
    References: https://en.wikipedia.org/wiki/Ordinary_least_squares,
                https://en.wikipedia.org/wiki/Weighted_least_squares,
                https://en.wikipedia.org/wiki/Bayesian_linear_regression (see Posterior
                distribution, mu_n, Precision Matrix).

    The hack is to estimate the residual variance directly and to use that to
    back out the Lambda matrix
    The problem is that you need a beta to estimate the residual variance, so
    we do two passes, the first time with the prior beta

    :param x: K x T x N numeric array with the second dimension as time. K
              variables. N time series
              Note: x is not automatically augmented to contain an intercept term
    :param y: T x N array. Dependent variable
    :param prior_mean: K vector, prior for beta
    :param prior_covar: K x K matrix, covariance matrix for beta estimates
    :param decay_scale: number of time steps each to decay by e, OR a length-K vector
                        of per-feature decay scales
    :param y_decay_scale: optional single scalar decay for the y-side moments (sw, swy2).
                          If None, defaults to the scalar decay or the first element of the
                          decay vector.
    :return: array of betas of shape K x T x N
    """
    for arr in (x, y, prior_mean, prior_covar):
        assert isinstance(arr, np.ndarray)
        assert arr.dtype.kind in "fdi"
        assert not np.any(np.isinf(arr))

    assert x.ndim == 3
    K, T, N = x.shape
    assert y.shape == (T, N)
    assert prior_mean.shape == (K,)
    assert prior_covar.shape == (K, K)
    assert qarray.is_positive_definite(prior_covar)

    # --- BEGIN added logic for vector decay + optional y decay ---
    if np.isscalar(decay_scale):
        assert isinstance(decay_scale, Real)
        assert qarray.gt(decay_scale, 0)
        decay = np.exp(-1 / decay_scale)     # scalar decay (original behavior)
        decay_vec = None                     # marks scalar path for x-side
        decay_outer = decay                  # scalar
    else:
        decay_scale = np.asarray(decay_scale, dtype=DT_DOUBLE)
        assert decay_scale.shape == (K,)
        assert np.all(qarray.gt(decay_scale, 0))
        decay_vec = np.exp(-1 / decay_scale)  # (K,)
        decay = decay_vec[0]                  # fallback scalar for y-side if needed
        if np.allclose(decay_vec, decay_vec[0]):
            decay_outer = decay_vec[0]        # scalar fast path
            decay_vec = None                  # treat as scalar for the loop
        else:
            decay_outer = np.sqrt(decay_vec[:, None] * decay_vec[None, :])  # K x K

    if y_decay_scale is None:
        y_decay = decay  # use scalar x-decay (or first element of vector) for y-side
    else:
        assert isinstance(y_decay_scale, Real)
        assert qarray.gt(y_decay_scale, 0)
        y_decay = np.exp(-1 / y_decay_scale)
    # --- END added logic ---

    prior_precision = np.linalg.inv(prior_covar)

    # Weight NaN data points as zero
    bad_x = np.isnan(x)
    bad = bad_x.any(axis=0) | np.isnan(y)
    w = (~bad).astype(DT_DOUBLE)

    # Replace x and y with finite versions.
    x0 = qarray.set_array(x, bad_x, 0)
    y0 = qarray.set_array(y, bad, 0)

    # Keep track of moments across time.
    sw, swy2 = (np.zeros((N,), DT_DOUBLE) for _ in range(2))
    swx2 = np.zeros((K, K, N), DT_DOUBLE)
    swxy = np.zeros((K, N), DT_DOUBLE)

    beta = np.empty(x.shape, DT_DOUBLE)

    # template for filling in betas
    beta_init = qarray.set_array(np.empty((N, K), DT_DOUBLE), np.s_[...], prior_mean)

    def iterate_beta(beta_a: np.ndarray) -> np.ndarray:
        """
        Using the given beta, estimate the residual variance and use this to
        compute a posterior beta from the given prior
        :param beta_a: N x K
        :return:
        """
        xy = swxy.T[:, None, :]     # N x 1 x K
        xx = swx2.T                 # N x K x K
        bt = beta_a[:, :, None]     # N x K x 1
        btt = beta_a[:, None, :]    # N x 1 x K

        # Use this beta to estimate the residual variance
        with np.errstate(invalid='ignore'):
            res_var0 = (swy2 + (-2 * (xy @ bt) + btt @ xx @ bt)[:, 0, 0]) / sw

        # It should always be non-negative
        assert np.all(np.isnan(res_var0) | qarray.geq(res_var0, 0))
        good = np.nonzero(qarray.geq(res_var0, 0))[0]

        # We add a bit to prevent zero residual variance
        # The reason is that we may have a few data points and therefore
        # singular XX^T matrix
        res_var = qarray.set_array(res_var0, np.isclose(res_var0, 0), 1e-5)

        # Compute precision matrix and recompute the beta with prior.
        lambda_ = res_var[:, None, None] * prior_precision   # N x K x K
        a = swx2.T + lambda_                                 # N x K x K
        b = swxy.T + lambda_ @ prior_mean                    # N x K
        return qarray.set_array(beta_a, good,
                                np.linalg.solve(a[good], b[good]))

    for t in range(T):
        # Update moments.
        sw   = sw   * y_decay + w[t]                        # N      (changed: y_decay)
        swy2 = swy2 * y_decay + w[t] * np.square(y0[t])     # N      (changed: y_decay)

        if decay_vec is None:
            # scalar path (original syntax)
            swxy = swxy * decay + w[t] * y0[t] * x0[:, t]     # K x N
            swx2 = swx2 * decay + w[t] * x0[:, t][:, None] * x0[:, t][None, :]  # K x K x N
        else:
            # per-feature decay for x-side terms
            swxy = swxy * decay_vec[:, None] + w[t] * y0[t] * x0[:, t]          # K x N
            swx2 = swx2 * decay_outer[:, :, None] + w[t] * x0[:, t][:, None] * x0[:, t][None, :]  # K x K x N

        # Compute new posterior beta from old a couple of times.
        beta0 = iterate_beta(beta_init)
        beta1 = iterate_beta(beta0)
        beta[:, t] = beta1.T

    return beta


In [ ]:
import os
import random
from typing import Dict, Tuple, List

import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_error

# =============================================================================
# deterministic utilities -----------------------------------------------------
# =============================================================================
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)


# =============================================================================
# expanding-window CV ---------------------------------------------------------
# =============================================================================
def expanding_splits(
    n_samples: int,
    init_train_window: int,
    test_window: int
):
    """Yield (train_idx, val_idx) for an expanding window CV."""
    train_end = init_train_window
    while train_end + test_window <= n_samples:
        yield (
            np.arange(0, train_end),
            np.arange(train_end, train_end + test_window)
        )
        train_end += test_window


# =============================================================================
# main optimiser --------------------------------------------------------------
# =============================================================================
def optimize_lgbm_regressor_cv(
    X: pd.DataFrame,
    y: pd.Series,
    n_trials: int = 100,
    random_state: int = 42,
    init_train_window: int = 14,
    test_window: int = 14,
) -> Tuple[Dict, float, lgb.LGBMRegressor]:
    """
    Returns (best_params, best_rmse, fitted_model) using a robust
    'median-of-top-10' parameter selection strategy.
    """

    # -------------------------------------------------------------------------
    # 1. Pre-compute CV folds once (same for every trial)  # ==== NEW ====
    # -------------------------------------------------------------------------
    folds: List[Tuple[np.ndarray, np.ndarray]] = list(
        expanding_splits(len(X), init_train_window, test_window)
    )

    # -------------------------------------------------------------------------
    # 2. Objective fed to Optuna
    # -------------------------------------------------------------------------
    def objective(trial: optuna.trial.Trial) -> float:
        params = {
            "objective": "regression",
            "metric": "rmse",
            "boosting_type": "gbdt",
            "verbosity": -1,
            "random_state": random_state,
            # ----- search space -----
            "num_leaves": trial.suggest_int("num_leaves", 20, 150),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "n_estimators": trial.suggest_int("n_estimators", 50, 500),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.3, 0.8),
            "subsample_freq": 1,
            "feature_fraction": trial.suggest_float("feature_fraction", 0.3, 0.8),
            "min_child_samples": trial.suggest_int("min_child_samples", 50, 300),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.01, 10.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 10.0, log=True),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.2, log=True),
        }

        model = lgb.LGBMRegressor(**params)

        fold_rmses = []
        for tr_idx, val_idx in folds:
            X_tr, y_tr = X.iloc[tr_idx], y.iloc[tr_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

            model.fit(
                X_tr,
                y_tr,
                eval_set=[(X_val, y_val)],
                eval_metric="rmse",
                verbose=False,
                callbacks=[lgb.early_stopping(25, verbose=False)],
            )
            preds = model.predict(X_val, num_iteration=model.best_iteration_)
            rmse = mean_squared_error(y_val, preds, squared=False)
            fold_rmses.append(rmse)

        return float(np.mean(fold_rmses))

    # -------------------------------------------------------------------------
    # 3. Run Optuna
    # -------------------------------------------------------------------------
    sampler = optuna.samplers.TPESampler(seed=random_state)
    study = optuna.create_study(direction="minimize", sampler=sampler)
    study.optimize(objective, n_trials=n_trials)

    # -------------------------------------------------------------------------
    # 4. Robust selection: median of the top-10 trials    # ==== NEW ====
    # -------------------------------------------------------------------------
    def eval_cv(params: Dict) -> float:
        """Deterministic single-seed CV rescore using fixed folds."""
        params = params.copy()
        params.update(
            {
                "objective": "regression",
                "metric": "rmse",
                "verbosity": -1,
                "random_state": random_state,
            }
        )

        model = lgb.LGBMRegressor(**params)
        rmses = []
        for tr_idx, val_idx in folds:
            X_tr, y_tr = X.iloc[tr_idx], y.iloc[tr_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

            model.fit(
                X_tr,
                y_tr,
                eval_set=[(X_val, y_val)],
                eval_metric="rmse",
                verbose=False,
                callbacks=[lgb.early_stopping(25, verbose=False)],
            )
            preds = model.predict(X_val, num_iteration=model.best_iteration_)
            rmses.append(mean_squared_error(y_val, preds, squared=False))
        return float(np.mean(rmses))

    # ---- grab top-10 trials by Optuna's internal score
    top10 = sorted(study.trials, key=lambda t: t.value)[:10]

    # ---- re-score them deterministically
    rescored = []
    for tr in top10:
        rmse = eval_cv(tr.params)
        rescored.append((rmse, tr.params))
    rescored.sort(key=lambda x: x[0])  # lower RMSE = better

    # ---- choose the *median* (rank-5) config
    robust_rank = len(rescored) // 2  # 0-indexed, so 5th of 10
    best_rmse, best_params = rescored[robust_rank]

    # -------------------------------------------------------------------------
    # 5. Fit final model on all data with robust params
    # -------------------------------------------------------------------------
    final_model = lgb.LGBMRegressor(**best_params)
    final_model.fit(X, y)

    # Optional: show what we picked
    print(f"Robust median-rank RMSE : {best_rmse:.5f}")
    print("Robust hyper-parameters :")
    for k, v in best_params.items():
        print(f"  {k:20s}: {v}")

    return best_params, best_rmse, final_model


# =============================================================================
# EXAMPLE USAGE ---------------------------------------------------------------
# =============================================================================
if __name__ == "__main__":
    # Dummy data just to illustrate; replace with your own
    X_demo = pd.DataFrame(
        np.random.randn(200, 20), columns=[f"f{i}" for i in range(20)]
    )
    y_demo = pd.Series(np.random.randn(200))

    best_params, best_rmse, model = optimize_lgbm_regressor_cv(
        X_demo,
        y_demo,
        n_trials=50,
        init_train_window=60,
        test_window=20,
    )


In [ ]:
"""
financial_time_series_feature_code.py
-------------------------------------
A (reasonably) comprehensive collection of technical feature builders for OHLCV
financial time series using pandas & numpy. You can pick and choose, or call
`run_all()` to generate a wide panel.

Assumptions:
- Input DataFrame `df` has a DateTime index and at least the columns:
  ['open', 'high', 'low', 'close', 'volume'] (case-insensitive accepted via params).
- All functions are vectorized (no lookahead). Outputs align with input index.
- Windows/parameters are customizable via function arguments.

Usage Example
-------------
>>> import pandas as pd
>>> from financial_time_series_feature_code import FeatureGenerator
>>> df = pd.read_csv('prices.csv', parse_dates=['date'], index_col='date')
>>> fg = FeatureGenerator(df)
>>> features = fg.run_all()
>>> features.tail()

You can also call individual helpers like:
>>> features = fg.add_returns().add_moving_averages().df

Dependencies: pandas, numpy, scipy(optional for entropy). Install ta-lib if you
want to extend with exotic indicators not included here.

Author: (your name)
License: MIT
"""

from __future__ import annotations
import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from typing import Iterable, Dict, List, Optional, Tuple

try:
    from scipy.stats import entropy
except ImportError:
    entropy = None  # Some features will check this

# -----------------------------------------------------------------------------
# Utility functions
# -----------------------------------------------------------------------------

def _safe_col(df: pd.DataFrame, name: str, default: float = np.nan) -> pd.Series:
    """Return df[name] if exists else a Series of default values."""
    return df[name] if name in df.columns else pd.Series(default, index=df.index)


def rolling_z(series: pd.Series, window: int, ddof: int = 0) -> pd.Series:
    m = series.rolling(window).mean()
    s = series.rolling(window).std(ddof=ddof)
    return (series - m) / s


def crossover_flag(fast: pd.Series, slow: pd.Series) -> pd.Series:
    diff = fast - slow
    return np.sign(diff).diff().fillna(0)


def time_since_condition(cond: pd.Series, max_lookback: Optional[int] = None) -> pd.Series:
    """Bars since last True. cond is boolean Series. Vectorized implementation."""
    idx = np.where(cond.values, np.arange(len(cond)), np.nan)
    # forward fill last true index
    last = pd.Series(idx, index=cond.index).ffill()
    dist = np.arange(len(cond)) - last
    if max_lookback is not None:
        dist = np.minimum(dist, max_lookback)
    return pd.Series(dist, index=cond.index)


def kurtosis(series: pd.Series, window: int) -> pd.Series:
    return series.rolling(window).kurt()


def skewness(series: pd.Series, window: int) -> pd.Series:
    return series.rolling(window).skew()


def pct_rank(series: pd.Series, window: int) -> pd.Series:
    return series.rolling(window).apply(lambda x: x.rank().iloc[-1] / len(x), raw=False)


def hurst_exponent(series: pd.Series, window: int) -> pd.Series:
    """Approximate rolling Hurst exponent using the R/S method.
    Not super fast; use sparingly or vectorize further if needed."""
    def _hurst(x: np.ndarray) -> float:
        if len(x) < 20 or np.std(x) == 0:
            return np.nan
        y = x - x.mean()
        z = np.cumsum(y)
        R = z.max() - z.min()
        S = y.std()
        if S == 0:
            return np.nan
        return np.log(R / S) / np.log(len(x))
    return series.rolling(window).apply(lambda arr: _hurst(arr.values), raw=False)


def approximate_entropy(series: pd.Series, window: int, m: int = 2, r: float = 0.2) -> pd.Series:
    """Approximate entropy (ApEn). Needs scipy for norms? We'll use numpy.
    r is tolerance * std within the window. Implement simple version."""
    def _apen(x: np.ndarray, m: int, r: float) -> float:
        N = len(x)
        if N <= m + 1:
            return np.nan
        std_x = np.std(x)
        if std_x == 0:
            return np.nan
        r_scaled = r * std_x
        def _phi(m):
            patterns = np.array([x[i:i+m] for i in range(N - m + 1)])
            C = []
            for p in patterns:
                dist = np.max(np.abs(patterns - p), axis=1)
                C.append(np.mean(dist <= r_scaled))
            return np.mean(np.log(C + np.finfo(float).eps))
        return _phi(m) - _phi(m + 1)
    return series.rolling(window).apply(lambda arr: _apen(arr, m, r), raw=False)


def rolling_entropy(series: pd.Series, window: int, bins: int = 20) -> pd.Series:
    if entropy is None:
        return pd.Series(np.nan, index=series.index)
    def _roll_ent(x):
        hist, _ = np.histogram(x, bins=bins)
        return entropy(hist + 1e-12)
    return series.rolling(window).apply(_roll_ent, raw=False)


# Range-based volatility estimators ------------------------------------------------

def parkinson_vol(high: pd.Series, low: pd.Series, window: int) -> pd.Series:
    rs = (np.log(high / low)) ** 2
    return np.sqrt(rs.rolling(window).mean() / (4 * np.log(2)))


def garman_klass_vol(open_: pd.Series, high: pd.Series, low: pd.Series, close: pd.Series, window: int) -> pd.Series:
    log_hl = np.log(high / low)
    log_co = np.log(close / open_)
    rs = 0.5 * (log_hl ** 2) - (2 * np.log(2) - 1) * (log_co ** 2)
    return np.sqrt(rs.rolling(window).mean())


def rogers_satchell_vol(open_, high, low, close, window: int) -> pd.Series:
    rs = np.log(high / close) * np.log(high / open_) + np.log(low / close) * np.log(low / open_)
    return np.sqrt(rs.rolling(window).mean())


def yang_zhang_vol(open_, high, low, close, prev_close, window: int) -> pd.Series:
    log_oc = np.log(open_ / prev_close)
    log_co = np.log(close / open_)
    log_hl = np.log(high / low)
    # Using coefficients from original paper
    k = 0.34 / (1.34 + (window + 1) / (window - 1))
    vol_open = log_oc.rolling(window).var()
    vol_close = log_co.rolling(window).var()
    vol_range = log_hl.rolling(window).mean() * (np.pi ** 2 / 8)
    return np.sqrt(vol_open + k * vol_close + (1 - k) * vol_range)


# -----------------------------------------------------------------------------
# Core Feature Generator
# -----------------------------------------------------------------------------

@dataclass
class FeatureGenerator:
    df: pd.DataFrame
    open_col: str = 'open'
    high_col: str = 'high'
    low_col: str = 'low'
    close_col: str = 'close'
    volume_col: str = 'volume'
    # internal
    _features_added: List[str] = field(default_factory=list)

    @property
    def px(self) -> pd.Series:
        return self.df[self.close_col]

    def _register(self, cols: Iterable[str]):
        self._features_added.extend(cols)

    # ------------------------------------------------------------------
    # Basic price/return features
    # ------------------------------------------------------------------
    def add_returns(self, periods: Iterable[int] = (1, 5, 10), log: bool = True) -> 'FeatureGenerator':
        close = self.px
        for p in periods:
            if log:
                self.df[f'ret_log_{p}'] = np.log(close).diff(p)
            self.df[f'ret_{p}'] = close.pct_change(p)
        self._register([c for c in self.df.columns if c.startswith('ret_')])
        return self

    def add_ohlc_gaps_ranges(self) -> 'FeatureGenerator':
        o = self.df[self.open_col]
        h = self.df[self.high_col]
        l = self.df[self.low_col]
        c = self.df[self.close_col]
        self.df['hl_range'] = (h - l) / l
        self.df['co_return'] = (c - o) / o
        self.df['gap_pc'] = (o - c.shift(1)) / c.shift(1)
        self.df['gap_filled'] = ((c >= c.shift(1)) & (o < c.shift(1))) | ((c <= c.shift(1)) & (o > c.shift(1)))
        self.df['time_to_fill_gap'] = time_since_condition(self.df['gap_filled'])  # bars since filled
        self._register(['hl_range', 'co_return', 'gap_pc', 'gap_filled', 'time_to_fill_gap'])
        return self

    def add_rolling_stats(self, windows: Iterable[int] = (5, 20, 60)) -> 'FeatureGenerator':
        r = self.df['ret_log_1'] if 'ret_log_1' in self.df.columns else np.log(self.px).diff()
        for w in windows:
            self.df[f'roll_mean_ret_{w}'] = r.rolling(w).mean()
            self.df[f'roll_std_ret_{w}'] = r.rolling(w).std()
            self.df[f'roll_skew_ret_{w}'] = r.rolling(w).skew()
            self.df[f'roll_kurt_ret_{w}'] = r.rolling(w).kurt()
            self.df[f'roll_autocorr_ret_{w}'] = r.rolling(w).apply(lambda x: pd.Series(x).autocorr(lag=1), raw=False)
        self._register([c for c in self.df.columns if c.startswith('roll_')])
        return self

    # ------------------------------------------------------------------
    # Trend / Moving averages / Bands
    # ------------------------------------------------------------------
    def add_moving_averages(self, fast: int = 12, slow: int = 26, other_windows: Iterable[int] = (5, 20, 60)) -> 'FeatureGenerator':
        px = self.px
        self.df[f'ema_{fast}'] = px.ewm(span=fast, adjust=False).mean()
        self.df[f'ema_{slow}'] = px.ewm(span=slow, adjust=False).mean()
        self.df['ema_diff'] = self.df[f'ema_{fast}'] - self.df[f'ema_{slow}']
        self.df['ema_cross_flag'] = crossover_flag(self.df[f'ema_{fast}'], self.df[f'ema_{slow}'])
        for w in other_windows:
            self.df[f'sma_{w}'] = px.rolling(w).mean()
        self._register(['ema_diff', 'ema_cross_flag'] + [f'ema_{fast}', f'ema_{slow}'] + [f'sma_{w}' for w in other_windows])
        return self

    def add_bollinger_bands(self, window: int = 20, n_std: float = 2.0) -> 'FeatureGenerator':
        px = self.px
        sma = px.rolling(window).mean()
        std = px.rolling(window).std()
        upper = sma + n_std * std
        lower = sma - n_std * std
        self.df['bb_upper'] = upper
        self.df['bb_lower'] = lower
        self.df['bb_pctB'] = (px - lower) / (upper - lower)
        self.df['bb_width'] = (upper - lower) / sma
        self._register(['bb_upper', 'bb_lower', 'bb_pctB', 'bb_width'])
        return self

    def add_donchian_channels(self, window: int = 20) -> 'FeatureGenerator':
        h = self.df[self.high_col]
        l = self.df[self.low_col]
        self.df['donchian_upper'] = h.rolling(window).max()
        self.df['donchian_lower'] = l.rolling(window).min()
        self.df['donchian_width'] = (self.df['donchian_upper'] - self.df['donchian_lower']) / self.df['donchian_lower']
        self._register(['donchian_upper', 'donchian_lower', 'donchian_width'])
        return self

    # ------------------------------------------------------------------
    # Momentum / Oscillators
    # ------------------------------------------------------------------
    def add_rsi(self, period: int = 14) -> 'FeatureGenerator':
        px = self.px
        delta = px.diff()
        up = delta.clip(lower=0)
        down = -delta.clip(upper=0)
        roll_up = up.ewm(alpha=1/period, adjust=False).mean()
        roll_down = down.ewm(alpha=1/period, adjust=False).mean()
        rs = roll_up / roll_down
        self.df[f'rsi_{period}'] = 100 - (100 / (1 + rs))
        self._register([f'rsi_{period}'])
        return self

    def add_stochastic(self, k_period: int = 14, d_period: int = 3) -> 'FeatureGenerator':
        h = self.df[self.high_col]
        l = self.df[self.low_col]
        c = self.df[self.close_col]
        lowest_low = l.rolling(k_period).min()
        highest_high = h.rolling(k_period).max()
        k = 100 * (c - lowest_low) / (highest_high - lowest_low)
        d = k.rolling(d_period).mean()
        self.df[f'stoch_k_{k_period}'] = k
        self.df[f'stoch_d_{d_period}'] = d
        self._register([f'stoch_k_{k_period}', f'stoch_d_{d_period}'])
        return self

    def add_williams_r(self, period: int = 14) -> 'FeatureGenerator':
        h = self.df[self.high_col]
        l = self.df[self.low_col]
        c = self.df[self.close_col]
        highest_high = h.rolling(period).max()
        lowest_low = l.rolling(period).min()
        self.df[f'williamsR_{period}'] = -100 * (highest_high - c) / (highest_high - lowest_low)
        self._register([f'williamsR_{period}'])
        return self

    def add_cci(self, period: int = 20) -> 'FeatureGenerator':
        tp = (self.df[self.high_col] + self.df[self.low_col] + self.df[self.close_col]) / 3
        sma = tp.rolling(period).mean()
        mad = tp.rolling(period).apply(lambda x: np.mean(np.abs(x - x.mean())), raw=False)
        self.df[f'cci_{period}'] = (tp - sma) / (0.015 * mad)
        self._register([f'cci_{period}'])
        return self

    def add_macd(self, fast: int = 12, slow: int = 26, signal: int = 9) -> 'FeatureGenerator':
        px = self.px
        ema_fast = px.ewm(span=fast, adjust=False).mean()
        ema_slow = px.ewm(span=slow, adjust=False).mean()
        macd = ema_fast - ema_slow
        sig = macd.ewm(span=signal, adjust=False).mean()
        hist = macd - sig
        self.df[f'macd_{fast}_{slow}'] = macd
        self.df[f'macd_signal_{signal}'] = sig
        self.df[f'macd_hist_{fast}_{slow}_{signal}'] = hist
        self._register([f'macd_{fast}_{slow}', f'macd_signal_{signal}', f'macd_hist_{fast}_{slow}_{signal}'])
        return self

    def add_trix(self, period: int = 15) -> 'FeatureGenerator':
        px = self.px
        ema1 = px.ewm(span=period, adjust=False).mean()
        ema2 = ema1.ewm(span=period, adjust=False).mean()
        ema3 = ema2.ewm(span=period, adjust=False).mean()
        self.df[f'trix_{period}'] = ema3.pct_change()
        self._register([f'trix_{period}'])
        return self

    # ------------------------------------------------------------------
    # Volatility / Range
    # ------------------------------------------------------------------
    def add_volatility(self, windows: Iterable[int] = (10, 20, 60)) -> 'FeatureGenerator':
        r = self.df['ret_log_1'] if 'ret_log_1' in self.df.columns else np.log(self.px).diff()
        for w in windows:
            self.df[f'realized_vol_{w}'] = r.rolling(w).std()
        self._register([f'realized_vol_{w}' for w in windows])
        return self

    def add_atr(self, period: int = 14) -> 'FeatureGenerator':
        h = self.df[self.high_col]
        l = self.df[self.low_col]
        c = self.df[self.close_col]
        prev_c = c.shift(1)
        tr = pd.concat([(h - l), (h - prev_c).abs(), (l - prev_c).abs()], axis=1).max(axis=1)
        self.df[f'atr_{period}'] = tr.rolling(period).mean()
        self._register([f'atr_{period}'])
        return self

    def add_range_vol_estimators(self, window: int = 20) -> 'FeatureGenerator':
        o = self.df[self.open_col]
        h = self.df[self.high_col]
        l = self.df[self.low_col]
        c = self.df[self.close_col]
        prev_c = c.shift(1)
        self.df[f'parkinson_vol_{window}'] = parkinson_vol(h, l, window)
        self.df[f'gk_vol_{window}'] = garman_klass_vol(o, h, l, c, window)
        self.df[f'rs_vol_{window}'] = rogers_satchell_vol(o, h, l, c, window)
        self.df[f'yz_vol_{window}'] = yang_zhang_vol(o, h, l, c, prev_c, window)
        self._register([f'parkinson_vol_{window}', f'gk_vol_{window}', f'rs_vol_{window}', f'yz_vol_{window}'])
        return self

    # ------------------------------------------------------------------
    # Volume / Flow
    # ------------------------------------------------------------------
    def add_volume_features(self, windows: Iterable[int] = (5, 20, 60)) -> 'FeatureGenerator':
        v = self.df[self.volume_col]
        for w in windows:
            self.df[f'vol_mean_{w}'] = v.rolling(w).mean()
            self.df[f'vol_z_{w}'] = rolling_z(v, w)
            self.df[f'vol_pct_rank_{w}'] = v.rolling(w).apply(lambda x: x.rank().iloc[-1] / len(x), raw=False)
        self._register([c for c in self.df.columns if c.startswith('vol_')])
        return self

    def add_obv(self) -> 'FeatureGenerator':
        c = self.df[self.close_col]
        v = self.df[self.volume_col]
        direction = np.sign(c.diff()).fillna(0)
        self.df['obv'] = (direction * v).cumsum()
        self._register(['obv'])
        return self

    def add_adl_cmf_vpt(self, period: int = 20) -> 'FeatureGenerator':
        h = self.df[self.high_col]
        l = self.df[self.low_col]
        c = self.df[self.close_col]
        v = self.df[self.volume_col]
        mfm = ((c - l) - (h - c)) / (h - l).replace(0, np.nan)  # money flow multiplier
        mfv = mfm * v
        self.df['adl'] = mfv.cumsum()
        # Chaikin Money Flow
        self.df[f'cmf_{period}'] = mfv.rolling(period).sum() / v.rolling(period).sum()
        # Volume Price Trend
        self.df['vpt'] = ((c.pct_change()).fillna(0) * v).cumsum()
        self._register(['adl', f'cmf_{period}', 'vpt'])
        return self

    def add_vwap_distance(self, intraday: bool = False) -> 'FeatureGenerator':
        """VWAP requires intraday ticks or OHLCV with cumulative volume per day.
        If intraday=False, we approximate rolling VWAP using daily bars (coarse)."""
        c = self.df[self.close_col]
        v = self.df[self.volume_col]
        if intraday:
            # Expect df to have a 'session' column or reset each day externally.
            # Here we do a simple cumulative intraday VWAP per day assumption.
            day = self.df.index.date
            pv = (c * v).groupby(day).cumsum()
            vv = v.groupby(day).cumsum()
            vwap = pv / vv
        else:
            # Rolling VWAP over 20 bars as a fallback
            pv = (c * v).rolling(20).sum()
            vv = v.rolling(20).sum()
            vwap = pv / vv
        self.df['vwap'] = vwap
        self.df['vwap_distance'] = (c - vwap) / vwap
        self._register(['vwap', 'vwap_distance'])
        return self

    # ------------------------------------------------------------------
    # Cross-Asset / Relative / Correlation
    # ------------------------------------------------------------------
    def add_relative_strength(self, benchmark: pd.Series, windows: Iterable[int] = (20, 60)) -> 'FeatureGenerator':
        px = self.px
        rel = px / benchmark.reindex(px.index).ffill()
        self.df['rel_strength'] = rel
        for w in windows:
            self.df[f'rel_corr_{w}'] = px.rolling(w).corr(benchmark)
            # Beta via covariance / var
            cov = px.pct_change().rolling(w).cov(benchmark.pct_change())
            var = benchmark.pct_change().rolling(w).var()
            self.df[f'beta_{w}'] = cov / var
        self._register(['rel_strength'] + [f'rel_corr_{w}' for w in windows] + [f'beta_{w}' for w in windows])
        return self

    # ------------------------------------------------------------------
    # Seasonality / Calendar
    # ------------------------------------------------------------------
    def add_calendar_features(self) -> 'FeatureGenerator':
        idx = self.df.index
        if not isinstance(idx, pd.DatetimeIndex):
            raise ValueError('Index must be DatetimeIndex to add calendar features')
        self.df['dow'] = idx.dayofweek  # 0=Mon
        self.df['dom'] = idx.day
        self.df['month'] = idx.month
        self.df['quarter'] = idx.quarter
        # One-hot if needed by model
        dow_dummies = pd.get_dummies(self.df['dow'], prefix='dow')
        month_dummies = pd.get_dummies(self.df['month'], prefix='m')
        self.df = pd.concat([self.df, dow_dummies, month_dummies], axis=1)
        self._register(['dow', 'dom', 'month', 'quarter'] + list(dow_dummies.columns) + list(month_dummies.columns))
        return self

    # ------------------------------------------------------------------
    # Entropy / Fractal / Info-theoretic
    # ------------------------------------------------------------------
    def add_entropy_features(self, windows: Iterable[int] = (50,), bins: int = 20) -> 'FeatureGenerator':
        r = self.df['ret_log_1'] if 'ret_log_1' in self.df.columns else np.log(self.px).diff()
        for w in windows:
            self.df[f'entropy_{w}'] = rolling_entropy(r, w, bins)
        self._register([f'entropy_{w}' for w in windows])
        return self

    def add_hurst(self, windows: Iterable[int] = (100,)) -> 'FeatureGenerator':
        r = self.df['ret_log_1'] if 'ret_log_1' in self.df.columns else np.log(self.px).diff()
        for w in windows:
            self.df[f'hurst_{w}'] = hurst_exponent(r, w)
        self._register([f'hurst_{w}' for w in windows])
        return self

    def add_approx_entropy(self, windows: Iterable[int] = (100,), m: int = 2, r: float = 0.2) -> 'FeatureGenerator':
        rts = self.df['ret_log_1'] if 'ret_log_1' in self.df.columns else np.log(self.px).diff()
        for w in windows:
            self.df[f'apen_{w}'] = approximate_entropy(rts, w, m=m, r=r)
        self._register([f'apen_{w}' for w in windows])
        return self

    # ------------------------------------------------------------------
    # Event / Pattern-based (basic versions)
    # ------------------------------------------------------------------
    def add_breakout_flags(self, window: int = 20) -> 'FeatureGenerator':
        h = self.df[self.high_col]
        l = self.df[self.low_col]
        c = self.df[self.close_col]
        hh = h.rolling(window).max().shift(1)
        ll = l.rolling(window).min().shift(1)
        self.df[f'breakout_up_{window}'] = (c > hh).astype(int)
        self.df[f'breakout_dn_{window}'] = (c < ll).astype(int)
        self._register([f'breakout_up_{window}', f'breakout_dn_{window}'])
        return self

    def add_time_since_extrema(self, lookback: int = 60) -> 'FeatureGenerator':
        px = self.px
        rolling_max_idx = px.rolling(lookback).apply(lambda s: np.argmax(s), raw=False)
        rolling_min_idx = px.rolling(lookback).apply(lambda s: np.argmin(s), raw=False)
        self.df[f'ts_since_high_{lookback}'] = lookback - 1 - rolling_max_idx
        self.df[f'ts_since_low_{lookback}'] = lookback - 1 - rolling_min_idx
        self._register([f'ts_since_high_{lookback}', f'ts_since_low_{lookback}'])
        return self

    # ------------------------------------------------------------------
    # Risk / Performance Metrics (backward-looking)
    # ------------------------------------------------------------------
    def add_sharpe_like(self, window: int = 60) -> 'FeatureGenerator':
        r = self.df['ret_log_1'] if 'ret_log_1' in self.df.columns else np.log(self.px).diff()
        mean = r.rolling(window).mean()
        vol = r.rolling(window).std()
        self.df[f'sharpe_like_{window}'] = mean / vol
        self._register([f'sharpe_like_{window}'])
        return self

    def add_drawdown_stats(self, window: int = 252) -> 'FeatureGenerator':
        px = self.px
        roll_max = px.rolling(window, min_periods=1).max()
        dd = (px / roll_max - 1)
        self.df[f'drawdown_{window}'] = dd
        # Max drawdown in window
        self.df[f'max_drawdown_{window}'] = dd.rolling(window).min()
        self._register([f'drawdown_{window}', f'max_drawdown_{window}'])
        return self

    # ------------------------------------------------------------------
    # Convenience: Run many features
    # ------------------------------------------------------------------
    def run_all(self) -> pd.DataFrame:
        """Run a broad set. Modify to your taste. Returns the feature DataFrame."""
        return (self.add_returns()
                    .add_ohlc_gaps_ranges()
                    .add_rolling_stats()
                    .add_moving_averages()
                    .add_bollinger_bands()
                    .add_donchian_channels()
                    .add_rsi()
                    .add_stochastic()
                    .add_williams_r()
                    .add_cci()
                    .add_macd()
                    .add_trix()
                    .add_volatility()
                    .add_atr()
                    .add_range_vol_estimators()
                    .add_volume_features()
                    .add_obv()
                    .add_adl_cmf_vpt()
                    .add_vwap_distance(intraday=False)
                    .add_calendar_features()
                    .add_entropy_features()
                    .add_hurst()
                    .add_approx_entropy()
                    .add_breakout_flags()
                    .add_time_since_extrema()
                    .add_sharpe_like()
                    .add_drawdown_stats()
                    .df)


# If you want to run as a script for quick test
if __name__ == "__main__":
    # Quick demo with random data (for structure testing only)
    idx = pd.date_range('2020-01-01', periods=500, freq='D')
    rng = np.random.default_rng(42)
    price = 100 * np.exp(np.cumsum(rng.normal(0, 0.01, size=len(idx))))
    high = price * (1 + rng.uniform(0, 0.02, size=len(idx)))
    low = price * (1 - rng.uniform(0, 0.02, size=len(idx)))
    open_ = price * (1 + rng.normal(0, 0.002, size=len(idx)))
    close = price
    volume = rng.integers(1e5, 2e5, size=len(idx))
    df_demo = pd.DataFrame({'open': open_, 'high': high, 'low': low, 'close': close, 'volume': volume}, index=idx)

    fg = FeatureGenerator(df_demo)
    feat = fg.run_all()
    print(feat.tail())
